In [7]:
import os
import sys
import glob
from langchain_community.vectorstores import Chroma
from IPython.display import display, Markdown
from langchain.text_splitter import RecursiveCharacterTextSplitter
import warnings
warnings.filterwarnings('ignore')

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(kit_dir, ".."))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

from src.routing import Router
from enterprise_knowledge_retriever.src.document_retrieval import DocumentRetrieval

CONFIG_PATH = os.path.join(kit_dir,'config.yaml')

# init router

In [8]:
router = Router(CONFIG_PATH)

# init RAG

In [9]:
folder_loc = os.path.join(kit_dir,'data/')
docx_files = list(glob.glob(f'{folder_loc}/*.txt'))
docs = []
for doc in docx_files:
    with open(doc) as f:
        docs.append(f.read())
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 800,
        chunk_overlap  = 20,
        length_function = len,
        add_start_index = True,
        separators = ["\n\n\n","\n\n", "\n", "*", "."]
    )
text_chunks = text_splitter.create_documents(docs)
document_retrieval =  DocumentRetrieval()
embeddings = document_retrieval.load_embedding_model()
vectorstore = document_retrieval.create_vector_store(text_chunks, embeddings, output_db="data/vectordb/")
document_retrieval.init_retriever(vectorstore)
conversation = document_retrieval.get_qa_retrieval_chain()

2024-09-23 16:13:33,100 [INFO] - Load pretrained SentenceTransformer: intfloat/e5-large-v2


load INSTRUCTOR_Transformer


2024-09-23 16:13:37,244 [INFO] - Use pytorch device: cpu
2024-09-23 16:13:37,245 [INFO] - This is the collection name: collection_b069af78-72b2-4bfb-af70-cc80159b693f


max_seq_length  512
Collection name is None


2024-09-23 16:13:39,275 [INFO] - Vector store saved to data/vectordb/


# user input

In [10]:
query = "How to use sambatune?"
# query = "What is LLM?"

# pipeline
The router returns the correct datasource based on the user's query and then uses the appropriate pipeline to answer the query.

In [11]:
datasource = router.routing(query)
if datasource == "vectorstore":
    output = conversation.invoke({"question":query})
    response = output['answer']
else:
    response = document_retrieval.llm(query)

print(response)
print("--------------------------")
print(f"datasource: {datasource}")

To use SambaTune, you need to run it with the application YAML file as input. Here are the steps:

1. First, run `sambatune` and pass in a YAML file for your model. You can see an example of how to do this in the "Run the sample application" section.
2. Then, run the `sambatune_ui` command.
3. Finally, you can explore the results of the SambaTune run in the SambaTune GUI.

You can also run SambaTune with specific modes, such as instrument-only mode or all modes, by using the `--modes` option. For example, to run in instrument-only mode, you can use the command `sambatune --modes instrument -- /path/to/config.yaml`.

You can also specify other arguments, such as `model-args`, `compile-args`, and `run-args`, to customize the run. You can see a list of all options by running `sambatune --help`.

It's worth noting that you need to install the SambaTune GUI on the client system where the web browser runs, and you can get client install instructions from SambaNova customer support through th